# STOCKS PERFORMANCE PREDICTOR


This notebook contains all the steps required to build a dataset of financial data for a lot of stocks and to analyze it with different machine learning models. The objective is to find stocks that will grow in value in the future. 

This notebook does not leverage historic ticker price data, but financial indicators found in the 10-K filings that each publicly traded company releases yearly.

See the README.md for the background information. All packages used are easily retrievable and can be installed with either `pip` or `conda` depending on your Python setup. I used `Python 3.7.5`.

An internet connection is required in order to scrape the data from the web (we will use https://financialmodelingprep.com and `pandas_datareader`).

See tutorial folder for a guided example regarding the second part of this notebook.

Let's begin from the required imports.

In [ ]:
from sys import stdout
import numpy as np
import pandas as pd
from pandas_datareader import data
import json

# Reading data from external sources
import urllib as u
from urllib.request import urlopen

# Machine learning (preprocessing, models, evaluation)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.metrics import classification_report

# Graphics
from tqdm import tqdm

These are three helper functions that will be used throughout this notebook.

`get_json_data`: used to scrape the links to financialmodelingprep API.

`get_price_var`: used to compute the price variation during 2019, leverages `pandas_datareader` with Yahoo.

`find_in_json`: used to scan a json file for a key and finding its value.

In [ ]:
def get_json_data(url):
    '''
    Scrape data (which must be json format) from given url
    Input: url to financialmodelingprep API
    Output: json file
    '''
    response = urlopen(url)
    dat = response.read().decode('utf-8')
    return json.loads(dat)

def get_price_var(symbol):
    '''
    Get historical price data for a given symbol leveraging the power of pandas_datareader and Yahoo.
    Compute the difference between first and last available time-steps in terms of Adjusted Close price..
    Input: ticker symbol
    Output: price variation 
    '''
    # read data
    prices = data.DataReader(symbol, 'yahoo', '2019-01-01', '2019-12-31')['Adj Close']

    # get all timestamps for specific lookups
    today = prices.index[-1]
    start = prices.index[0]

    # calculate percentage price variation
    price_var = ((prices[today] - prices[start]) / prices[start]) * 100
    return price_var

def find_in_json(obj, key):
    '''
    Scan the json file to find the value of the required key.
    Input: json file
           required key
    Output: value corresponding to the required key
    '''
    # Initialize output as empty
    arr = []

    def extract(obj, arr, key):
        '''
        Recursively search for values of key in json file.
        '''
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

## DATA PART I: LIST OF STOCKS


First, we need to get a list of stocks that will be used to build the dataset. Since there are thousands of stocks whose information can be scraped online, I decided to simply pull the whole list of available stocks on Financial Modeling Prep API. 

The list comprehends a total of more than 7k stocks, which clearly spans more than one sector. Indeed, each company belongs to its sector (Technology, Healthcare, Energy, ...), which in turn may be characterized by certain seasonalities, macro-economic trends and so on. As of now, I decided to focus on the Technology sector: this means that from the complete list of available stocks `available_tickers` I only keep those whose sector is equal to `Technology`. This operation is quite straight forward thanks to the power of `pandas` library.

So, the list `tickers_tech` will contain all the available stocks, on Financial Modeling Prep API, belonging to the Technology sector.

In [ ]:
url = 'https://financialmodelingprep.com/api/v3/company/stock/list'
ticks_json = get_json_data(url)
available_tickers = find_in_json(ticks_json, 'symbol')

tickers_sector = []
for tick in tqdm(available_tickers):
    url = 'https://financialmodelingprep.com/api/v3/company/profile/' + tick # get sector from here
    a = get_json_data(url)
    tickers_sector.append(find_in_json(a, 'sector'))

S = pd.DataFrame(tickers_sector, index=available_tickers, columns=['Sector'])

# Get list of tickers from TECHNOLOGY sector
tickers_tech = S[S['Sector'] == 'Technology'].index.values.tolist()

## DATA PART II: GET PRICE VARIATION THROUGHOUT 2019


The price variation of each stock listed in `tickers_tech` during 2019 will be used as metric to distinguish between stocks worth buying and those that are not. So, we need to:
  * pull all the **daily Adjusted Close Price** for each stock, compute difference (this is done thanks to the helper function `get_price_var`
  * if no data is found, skip the stock
  * limit the number of stocks to be scanned to 1000
  * store stocks and relative 2019 price variations in the dataframe `D`

In [ ]:
pvar_list, tickers_found = [], []
num_tickers_desired = 1000
count = 0
tot = 0
TICKERS = tickers_tech

for ticker in TICKERS:
    tot += 1 
    try:
        pvar = get_price_var(ticker)
        pvar_list.append(pvar)
        tickers_found.append(ticker)
        count += 1
    except:
        pass

    stdout.write(f'\rScanned {tot} tickers. Found {count}/{len(TICKERS)} usable tickers (max tickets = {num_tickers_desired}).')
    stdout.flush()

    if count == num_tickers_desired: # if there are more than 1000 tickers in sectors, stop
        break

# Store everything in a dataframe
D = pd.DataFrame(pvar_list, index=tickers_found, columns=['2019 PRICE VAR [%]'])

For the stocks in `D`, we now need to find the values of the indicators that will become the input data to the classification models. We leverage once again the FinancialModelingPrep API.

First we load the `indicators.tx` file (available in the repository). As explained the the `README` document, a plethora of financial indicators are being scraped. I decided to perform a **brute force** of all the available indicators from the FinancialModelingPrep API, and then I will worry about cleaning and preparing the dataset for the models. The table below summarizes the quantity of financial indicator available for each category.


||Income Statement|Balance Sheet Statement|Cash Flow Statement|Financial Ratios|Key Metrics|Financial Growth|
|:-------|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
|Quantity| 31 | 29 | 15 | 59 | 57 | 33 |


In total, 224 indicators are available. However, since there are some duplicates, the actual number of indicators in `indicators.txt` is 221 (not counting the date).

In [ ]:
# Load financial indicators from the provided .txt file
indicators = []
filename = 'indicators.txt'
with open(filename, 'r') as f:
    for line in f:
        indicators.append(line.strip('\n'))

## DATA PART III: SCRAPE FINANCIAL INDICATORS AND BUILD RAW DATASET


As of now we have listed the stocks that belong to the Technology sector, and we have also listed their 2019 price variation. It is time to scrape the financial indicators that will later be used as input features to out classification models.

The scraping will once again be performed thanks to the Financial Modeling Prep API. This process is quite time-consuming since it is required to pull a lot of data iteratively (due to a limit on the number of batch requests that can be performed with this API).

Furthermore, it is important to keep in mind that:
  * it is required to pull data within a specific time frame. Since the objective is the classification of a stock according to its price variation during year 2019, the financial indicators must belong to the end of year 2018. Historic financial data is an option that is considered as future development.
  * it is possible, albeit uncommon, that one company filed two 10-K documents in the same year. In this case only the most recent entry must be kept.
  * it is possible that the API does not return any data at all for a given stock. In this case the stock must be discarded.
  * not all indicators will return a value. It is fair to expect that a percentage of the indicators are missing for one reason or another. In this case, `np.nan` will be assigned to the missing entries, and we will take care of them in the cleaning stage.

In the end, what we want to obtain is a dataframe `DATA` where the rows correspond to the stocks for which the data has been found (`actual_tickers`) and the columns correspond to the financial indicators (`indicators`).

In [ ]:
# Initialize lists and dataframe (dataframe is a 2D numpy array filled with 0s)
missing_tickers, missing_index = [], []
d = np.zeros((len(tickers_found), len(indicators)))

for t, _ in enumerate(tqdm(tickers_found)):
    # Scrape indicators from financialmodelingprep API
    url0 = 'https://financialmodelingprep.com/api/v3/financials/income-statement/' + tickers_found[t]
    url1 = 'https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/' + tickers_found[t]
    url2 = 'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/' + tickers_found[t]
    url3 = 'https://financialmodelingprep.com/api/v3/financial-ratios/' + tickers_found[t]
    url4 = 'https://financialmodelingprep.com/api/v3/company-key-metrics/' + tickers_found[t]
    url5 = 'https://financialmodelingprep.com/api/v3/financial-statement-growth/' + tickers_found[t]
    a0 = get_json_data(url0)
    a1 = get_json_data(url1)
    a2 = get_json_data(url2)
    a3 = get_json_data(url3)
    a4 = get_json_data(url4)
    a5 = get_json_data(url5)
    
    # Combine all json files in a list, so that it can be scanned quickly
    A = [a0, a1, a2 , a3, a4, a5]
    all_dates = find_in_json(A, 'date')

    check = [s for s in all_dates if '2018' in s] # find all 2018 entries in dates
    if len(check) > 0:
        date_index = all_dates.index(check[0]) # get most recent 2018 entries, if more are present

        for i, _ in enumerate(indicators):
            ind_list = find_in_json(A, indicators[i])
            try:
                d[t][i] = ind_list[date_index]
            except:
                d[t][i] = np.nan # in case there is no value inserted for the given indicator

    else:
        missing_tickers.append(tickers_found[t])
        missing_index.append(t)

actual_tickers = [x for x in tickers_found if x not in missing_tickers]
d = np.delete(d, missing_index, 0)
DATA = pd.DataFrame(d, index=actual_tickers, columns=indicators) # raw dataset

## DATA PART IV: DATASET CLEANING & PREPARATION


The preparation of the dataset is somewhat an art form. I limited my actions to the application of the common practices, such as:

  * removing columns that have a lot of `nan` values.
  * removing columns that have a lot of `0` values.
  * fill the remaining `nan` values with the average value of the column.

For instance, in this specific case there are an average of 84 0-values per column, with a standard deviation of 140. So I decided to remove from the dataframe all those columns where the occurrences of 0-values is higher than 20 (20 being about 3.1% of the total number of rows of the dataset).

At the same time, there is an average of about 37 `nan` entries per column, with a standard deviation of circa 86. So I decided to remove from the dataframe all those columns where the occurrences of `nan` entries is higher than 15 (15 being about 2.4% of the total number of rows of the dataset). Then, the remaining `nan` entries have been filled with the average value of the column.

At the end of the cleaning process, the dataset `DATA` number of columns has decreased from 221 to 108, a 50% reduction. While certainly some of the discarded indicators were useless due to the lack of data, it is possible that useful data has been lost in this process. However, it must be considered that we need useful data across all stocks in the dataset, so I think it is acceptable to discard those indicators (columns) that may be relevant only to a subset of the dataset.


Finally, it is required to classify each sample. For each stock it has already be computed the difference in trading price between the first trading day on January 2019 and the last trading day on December 2019 (dataset `D`). If this difference is positive, then that stock will belong to class `1`, which is a **BUY** signal. On the contrary, if the difference in price is negative, the stock will be classified as a `0`, which is an **IGNORE** signal (do not buy). A quick recap is found in the table below. 


| 2019 PRICE VARIATION | CLASS | SIGNAL |
|:-----:|:-----:|:-----:|
| >= 0  | 1 | BUY |
| < 0   | 0 | IGNORE |


So, this array of `1` and `0` values will be appended as the last column of the dataframe `DATA`.

In [ ]:
# Remove columns that have more than 20 0-values
DATA = DATA.loc[:, DATA1.isin([0]).sum() <= 20]

# Remove columns that have more than 15 nan-values
DATA = DATA.loc[:, DATA1.isna().sum() <= 15]

# Fill remaining nan-values with column mean value
DATA = DATA.apply(lambda x: x.fillna(x.mean())) 

# Get price variation data only for tickers to be used
D2 = D.loc[DATA.index.values, :]

# Generate classification array
y = []
for i, _ in enumerate(D2.index.values):
    if D2.values[i] >= 0:
        y.append(1)
    else: 
        y.append(0)

# Add array to dataframe
DATA['class'] = y

## ML MODELS PART I: DATASET SPLIT AND STANDARDIZATION


In this second half of the notebook, the focus is shifted to the application of ML algorithms to classify the stocks collected into `DATA` in either BUY or IGNORE classes.

As explained in the `README` file, 4 models are currently implemented:
  * Support vector machine;
  * Random forest;
  * Extreme gradient boosting;
  * Multi-layer perceptron (feed-forward neural network).
  
  
Before training and testing them, it is required to perform a couple of pre-processing steps:
1. split `DATA` in train and testing datasets;
2. standardize each column of the dataset so that it has mean=0 and std=1.


For each ML model, grid search with cross-validation is run in order to find the optimum set of the main hyper-parameters, with the objective of maximizing the accuracy.


Finally, the models are compared with respect to the amount of $USD they'd return if a trader was to follow their predictions. The best model is the one that yields the highest gains.


So, let's begin with splitting the dataset `DATA` and allocating the classification column `DATA['class']` to the array of ground truth `y`.

In [ ]:
# Divide data in train and testing
train_split, test_split = train_test_split(DATA, test_size=0.2, random_state=1, stratify=df['class'])
X_train = train_split.iloc[:, :-1].values
y_train = train_split.iloc[:, -1].values
X_test = test_split.iloc[:, :-1].values
y_test = test_split.iloc[:, -1].values

Now, `StandardScaler()` is applied in order to standardize each column of both training and testing datasets. It is important to use the same coefficients when standardizing both datasets, hence it is leveraged the `.fit_transform()` method.

In [ ]:
# Standardize train/test datasets (use same coeff.)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## ML MODELS PART II: SVC


The main hyper-parameters are optimized via `GridSearchCV` method (with 5 cross-validation folds). A set of possible parameters is defined in the dictionary `tuned_parameters`. Pay attention to the `scoring='precision_weighted'` parameter used with `GridSearchCV`.

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}]

clf1 = GridSearchCV(SVC(random_state=1),
                    tuned_parameters,
                    n_jobs=6,
                    scoring='precision_weighted',
                    cv=5)
clf1.fit(X_train, y_train)

print('Best score, and parameters, found on development set:')
print()
print('%0.3f for %r' % (clf1.best_score_, clf1.best_params_))
print()

## ML MODELS PART III: RFC


The main hyper-parameters are optimized via `GridSearchCV` method (with 5 cross-validation folds). A set of possible parameters is defined in the dictionary `tuned_parameters`. Pay attention to the `scoring='precision_weighted'` parameter used with `GridSearchCV`.

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = {'n_estimators': [8, 32, 64, 128],
                    'max_features': ['auto', 'sqrt'],
                    'max_depth': [4, 5, 6, 7, 8],
                    'criterion': ['gini', 'entropy']}

clf2 = GridSearchCV(RandomForestClassifier(random_state=1),
                    tuned_parameters,
                    n_jobs=6,
                    scoring='precision_weighted',
                    cv=5)
clf2.fit(X_train, y_train)

print('Best score, and parameters, found on development set:')
print()
print('%0.3f for %r' % (clf2.best_score_, clf2.best_params_))
print()

## ML MODELS PART IV: XGB


The main hyper-parameters are optimized via `GridSearchCV` method (with 5 cross-validation folds). A set of possible parameters is defined in the dictionary `tuned_parameters`. Pay attention to the `scoring='precision_weighted'` parameter used with `GridSearchCV`.

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = {'learning_rate': [0.01, 0.001],
              'max_depth': [4, 5, 6, 7, 8],
              'n_estimators': [32, 256, 512, 1024]}

clf3 = GridSearchCV(xgb.XGBClassifier(random_state=1),
                   tuned_parameters,
                   n_jobs=6,
                   scoring='precision_weighted', 
                   cv=5)
clf3.fit(X_train, y_train)

print('Best score, and parameters, found on development set:')
print()
print('%0.3f for %r' % (clf3.best_score_, clf3.best_params_))
print()

## ML MODELS PART V: MLP


The main hyper-parameters are optimized via `GridSearchCV` method (with 5 cross-validation folds). A set of possible parameters is defined in the dictionary `tuned_parameters`. Due to the relatively small amount of data, `batch_size = 4`. Pay attention to the `scoring='precision_weighted'` parameter used with `GridSearchCV`.

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = {'hidden_layer_sizes': [(32,), (64,), (32, 64, 32)],
                    'activation': ['tanh', 'relu'],
                    'solver': ['lbfgs', 'adam']}

clf4 = GridSearchCV(MLPClassifier(random_state=1, batch_size=4, early_stopping=True), 
                    tuned_parameters,
                    n_jobs=6,
                    scoring='precision_weighted',
                    cv=5)

clf4.fit(X_train, y_train)

print('Best score, and parameters, found on development set:')
print()
print('%0.3f for %r' % (clf4.best_score_, clf4.best_params_))
print()

## ML MODELS PART VI: EVALUATION


To evaluate the performance of the models, it is not enough to rank and compare their accuracies. Indeed, being a financial application, it is desired to maximize the profit. So, the models' predicted classes are gathered in a dataset `df1` and:

  * if the predicted class = 1 --> Bob the trader buys $100 of that stock;
  * if the predicted class = 0 --> Bob the trader ignores the stock.
  

Consequetly, we evaluate the price difference for each stock, exploiting the dataset `D` that has been built in the first half of the notebook, which collects the percent price variation of all stocks. The buy orders, price variations, start and final values are all collected in the dataframe `df1`.

Since the dataframe `df1` is quite large, a more compact dataframe `MODELS_COMPARISON` is built in order to summarize the results so that it is easier to compare the performance of the ML models. The final row of `MODELS_COMPARISON` shows the net profit/loss for each ML model.

As a reference, it is also reported the percent gain/loss of both S&P 500 (^GSPC) and DOW JONES (^DJI), which are usually considered as benchmarks.

In [ ]:
# Get name of stocks that belong to testing dataset
tested_stocks = test_split.index.values

# Get 2019 price variation ONLY of tested stocks
df00 = D.loc[tested_stocks, :]

# Initial investment can be $100 for each stock whose predicted class = 1
buy_amount = 100

# In new dataframe df1, store all the information regarding each model's predicted class and relative gain/loss in $USD
df1 = pd.DataFrame(y_test, index=tested_stocks, columns=['ACTUAL']) # first column is the true class (BUY/INGORE)

df1['SVM'] = clf1.predict(X_test) # predict class for testing dataset
df1['VALUE START SVM [$]'] = df1['SVM'] * buy_amount # if class = 1 --> buy $100 of that stock
df1['VAR SVM [$]'] = (df00['2019 PRICE VAR [%]'].values / 100) * df1['VALUE START SVM [$]'] # compute price variation in $
df1['VALUE END SVM [$]'] = df1['VALUE START SVM [$]'] + df1['VAR SVM [$]'] # compute final value

df1['RF'] = clf2.predict(X_test)
df1['VALUE START RF [$]'] = df1['RF'] * buy_amount
df1['VAR RF [$]'] = (df00['2019 PRICE VAR [%]'].values / 100) * df1['VALUE START RF [$]']
df1['VALUE END RF [$]'] = df1['VALUE START RF [$]'] + df1['VAR RF [$]']

df1['XGB'] = clf3.predict(X_test)
df1['VALUE START XGB [$]'] = df1['XGB'] * buy_amount
df1['VAR XGB [$]'] = (df00['2019 PRICE VAR [%]'].values / 100) * df1['VALUE START XGB [$]']
df1['VALUE END XGB [$]'] = df1['VALUE START XGB [$]'] + df1['VAR XGB [$]']

df1['MLP'] = clf4.predict(X_test)
df1['VALUE START MLP [$]'] = df1['MLP'] * buy_amount
df1['VAR MLP [$]'] = (df00['2019 PRICE VAR [%]'].values / 100) * df1['VALUE START MLP [$]']
df1['VALUE END MLP [$]'] = df1['VALUE START MLP [$]'] + df1['VAR MLP [$]']

# Create a new, compact, dataframe in order to show gain/loss for each model
start_value_svm = df1['VALUE START SVM [$]'].sum()
final_value_svm = df1['VALUE END SVM [$]'].sum()
net_gain_svm = final_value_svm - start_value_svm
percent_gain_svm = net_gain_svm / start_value_svm

start_value_rf = df1['VALUE START RF [$]'].sum()
final_value_rf = df1['VALUE END RF [$]'].sum()
net_gain_rf = final_value_rf - start_value_rf
percent_gain_rf = net_gain_rf / start_value_rf

start_value_xgb = df1['VALUE START XGB [$]'].sum()
final_value_xgb = df1['VALUE END XGB [$]'].sum()
net_gain_xgb = final_value_xgb - start_value_xgb
percent_gain_xgb = net_gain_xgb / start_value_xgb

start_value_mlp = df1['VALUE START MLP [$]'].sum()
final_value_mlp = df1['VALUE END MLP [$]'].sum()
net_gain_mlp = final_value_mlp - start_value_mlp
percent_gain_mlp = net_gain_mlp / start_value_mlp

percent_gain_sp500 = get_price_var('^GSPC') # get percent gain of S&P500 index
percent_gain_dj = get_price_var('^DJI') # get percent gain of DOW JONES index
percent_gain_sector = D['2019 PRICE VAR [%]'].mean() # get percent gain of TECHNOLOGY sector

MODELS_COMPARISON = pd.DataFrame([start_value_svm, final_value_svm, net_gain_svm, percent_gain_svm],
                    index=['INITIAL COST [USD]', 'FINAL VALUE [USD]', 'GAIN/LOSS [USD]', 'GAIN/LOSS [%]'], columns=['SVM'])
MODELS_COMPARISON['RF'] = [start_value_rf, final_value_rf, net_gain_rf, percent_gain_rf]
MODELS_COMPARISON['XGB'] = [start_value_xgb, final_value_xgb, net_gain_xgb, percent_gain_xgb]
MODELS_COMPARISON['MLP'] = [start_value_mlp, final_value_mlp, net_gain_mlp, percent_gain_mlp]
MODELS_COMPARISON['S&P 500'] = ['', '', '', percent_gain_sp500]
MODELS_COMPARISON['DOW JONES'] = ['', '', '', percent_gain_dj]
MODELS_COMPARISON['SECTOR'] = ['', '', '', percent_gain_sector] 

For what concerns a more traditional comparison, it is also possible to print the `classification_report` for each ML model.

In [ ]:
print()
print(53 * '=')
print(15 * ' ' + 'SUPPORT VECTOR MACHINE')
print(53 * '-')
print(classification_report(y_test, clf1.predict(X_test), target_names=['IGNORE', 'BUY']))
print(53 * '-')
print(53 * '=')
print(20 * ' ' + 'RANDOM FOREST')
print(53 * '-')
print(classification_report(y_test, clf2.predict(X_test), target_names=['IGNORE', 'BUY']))
print(53 * '-')
print(53 * '=')
print(14 * ' ' + 'EXTREME GRADIENT BOOSTING')
print(53 * '-')
print(classification_report(y_test, clf3.predict(X_test), target_names=['IGNORE', 'BUY']))
print(53 * '-')
print(53 * '=')
print(15 * ' ' + 'MULTI-LAYER PERCEPTRON')
print(53 * '-')
print(classification_report(y_test, clf4.predict(X_test), target_names=['IGNORE', 'BUY']))
print(53 * '-')